In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd
import nltk
import seaborn as sns
import warnings
from textblob import TextBlob
import os
import json
import gzip
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
review_df = pd.read_csv('/content/gdrive/Shareddrives/RS/LA02/Reviews/reviews_merged_123456.csv', sep=',', engine='python')

In [ ]:
review_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,sentiment
0,5.0,HALİL TURGUT,Wc si temiz duzenli..,['Turkish Cuisine'],102194128241608748649,1342870724.0,"Jul 21, 2012",100000013500285534661,neutral
1,4.0,william spindler,Best War Wanton soup in Red Bluff,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,1394669496.0,"Mar 12, 2014",100000032416892623125,positive
2,5.0,william spindler,This is a review that is long overdo. I've bee...,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,1394826388.0,"Mar 14, 2014",100000032416892623125,positive
3,5.0,william spindler,"Some authentic rub BBQ, great food and don't m...",['Barbecue Restaurant'],111623070919810985923,1394671215.0,"Mar 12, 2014",100000032416892623125,positive
4,4.0,william spindler,Truly a Red Bluff standard. Great old fashione...,['Restaurant'],113854191152597312098,1394670357.0,"Mar 12, 2014",100000032416892623125,positive
...,...,...,...,...,...,...,...,...,...
6390571,2.0,charles mckinney,Too expensive for a fast food burger.,"['Hamburger Restaurant', 'Fast Food Restaurant']",116458473784504954830,1364706351.0,"Mar 30, 2013",118446742455312620560,negative
6390572,2.0,charles mckinney,Too many gangs frequent this place.,['Flea Market'],116649125736549598195,1367212161.0,"Apr 28, 2013",118446742455312620560,positive
6390573,1.0,charles mckinney,i will never go back. food and customer servic...,"['Buffet Restaurant', 'American Restaurant']",117952004983617019485,1315602371.0,"Sep 9, 2011",118446742455312620560,negative
6390574,2.0,Мартин Милчевски,Doesn't work.,['Post Office'],107994784120920658841,1343431770.0,"Jul 27, 2012",118446743512419806874,neutral


In [ ]:
review_df.shape

(6390576, 11)

In [ ]:
review_df = review_df.loc[~review_df['categories'].isnull()]

In [ ]:
review_df=review_df[review_df['sentiment']=='positive']

In [ ]:
review_df.shape

(4331954, 11)

In [ ]:
new_df = review_df[review_df['categories'].apply(lambda x: 'restaurant' in x.lower())]

In [ ]:
new_df.shape 

(1464077, 11)

In [ ]:
new_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,sentiment
1,4.0,william spindler,Best War Wanton soup in Red Bluff,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,1394669496.0,"Mar 12, 2014",100000032416892623125,positive
2,5.0,william spindler,This is a review that is long overdo. I've bee...,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,1394826388.0,"Mar 14, 2014",100000032416892623125,positive
3,5.0,william spindler,"Some authentic rub BBQ, great food and don't m...",['Barbecue Restaurant'],111623070919810985923,1394671215.0,"Mar 12, 2014",100000032416892623125,positive
4,4.0,william spindler,Truly a Red Bluff standard. Great old fashione...,['Restaurant'],113854191152597312098,1394670357.0,"Mar 12, 2014",100000032416892623125,positive
5,5.0,william spindler,"Long time favorite Mexican food, always consis...",['Mexican Restaurant'],115827996910815192564,1394669713.0,"Mar 12, 2014",100000032416892623125,positive
...,...,...,...,...,...,...,...,...,...
6390543,5.0,lou d,YOU want pasta and boyfriend wants man food. P...,"['Restaurant', 'European Restaurant', 'Pizza R...",102584205310037357459,1347086158.0,"Sep 7, 2012",118446727825246298271,positive
6390545,3.0,lou d,I've been coming to pats since I was an infant...,['Hamburger Restaurant'],117283431278933652539,1347085600.0,"Sep 7, 2012",118446727825246298271,positive
6390548,5.0,Justin McMinn,The best Hawaiian food in Denver. I recommend...,"['Barbecue Restaurant', 'Hawaiian Restaurant']",106541307972413987659,1369439617.0,"May 24, 2013",118446733868397970257,positive
6390557,1.0,charles mckinney,food is great but prices are a bit high.,"['Hamburger Restaurant', 'American Restaurant'...",105265629200019941559,1315601342.0,"Sep 9, 2011",118446742455312620560,positive


In [ ]:
# new_df.to_csv("/content/gdrive/Shareddrives/RS/LA02/toyset_restaurant.csv")